In [1]:
import seaborn as sns
import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from mpl_toolkits import mplot3d
from klepto.archives import dir_archive
import cv2
import time
import itertools
from scipy.io import savemat
import gc # to force remove objects' associated memory you deallocate using <<del var_name>>  or <<var_name = None>>

In [2]:
BEDFORMS = ['corals', 'canopy', 'rocks', 'dunes']
flow_speeds = ['fast', 'med', 'slow']
submergences = ['Deep', 'Intermed', 'Shallow']
colors = {'corals':'dodgerblue','canopy':'green', 'rocks':'orange', 'dunes':'brown'}
quantities = ['slope_Azi_angle', 'slope_x', 'slope_y'] # to save

In [3]:
# bed_idx = 3
# flow_idx = 2
# submergence_idx = 2
TEST = 2
DIST = 172
BATCH_FRAMES =  75# for data acquired at 3 fp, total of 900, per subfolder 225 frames. 225/75 = 9

# # SET the data variables
# FLOW_SPEED = flow_speeds[flow_idx]
# SUBMERGENCE = submergences[submergence_idx]
# BEDFORM = BEDFORMS[bed_idx]
# print("{}_{}Flow_{}H_test{}".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST))

## PSD using 2D FFT

In [4]:
s=[np.arange(len(BEDFORMS)), np.arange(len(flow_speeds)), np.arange(len(submergences)) ]
for bed_idx,flow_idx,submergence_idx in [(0,1,2), (1,1,2) (2,1,2), (3,1,2), (2,0,1)]:#list(itertools.product(*s)): 
    FLOW_SPEED = flow_speeds[flow_idx]
    SUBMERGENCE = submergences[submergence_idx]
    BEDFORM = BEDFORMS[bed_idx]
    print("---------------------*****{}_{}Flow_{}H****----------------------".format(BEDFORM, FLOW_SPEED, SUBMERGENCE))
#     try:
    for which_quantity in [0, 1, 2]:
        QUANTITY = quantities[which_quantity]
        print("--------------QUANTITY{}: {}---------".format(which_quantity,QUANTITY))
        for PORTION in [1,2,3,4]: # 4 subfolders split into 4 portions for saving
            print("#####PORTION: {}#####".format(PORTION))
            accumulated_data = []
            for serial,i in enumerate([PORTION]):#range(1,5):
                # choose the data
                SUBFOLDER = i

                # source data folders
#                 data_foldername = "waterjet_powerHighCapped1_Dist0cmBoilHeadtoROITop_15fps_25Pump_24cmH_industrialHose_subFolder{}".format(SUBFOLDER)
#                 data_foldername = "{}_{}Flow_{}H_test{}_centeredCam_{}cmDownstreamUpperROI_flume_LookAngle_35Deg_subFolder{}".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST,DIST, SUBFOLDER)
                data_foldername = "{}_{}Flow_{}H_test{}_subFolder{}".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST, SUBFOLDER)
                print(data_foldername)
                data_location =  '../TOSHIBA_Drive/saksham_polar_cam_FLIR'
                save_location =  "../TOSHIBA_Drive/saksham_polar_cam_FLIR/klepto_bulk_data_saves/all_MATdata_{}_{}Flow_{}H_test{}".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST)
#                 save_location =  "../TOSHIBA_Drive/saksham_polar_cam_FLIR/klepto_bulk_data_saves/all_MATdata_{}_{}Flow_{}H_test{}_centeredCam_{}cmDownstreamUpperROI/".format(BEDFORM, FLOW_SPEED, SUBMERGENCE, TEST, DIST)
#                 save_location =  "../TOSHIBA_Drive/saksham_polar_cam_FLIR/klepto_bulk_data_saves/" +\
#                                     "all_MATdata_waterjet_powerHighCapped1_Dist0cm_15fps_25Pump_24cmH"
                
                if not os.path.exists(save_location):
                    os.mkdir(save_location)
                    print("save folder location created.")
                klepto_saved_data_origin = 'klepto_bulk_data_saves' 

                # retrive the saved data
                data_origin = os.path.join(data_location, klepto_saved_data_origin, data_foldername)
                bed_data = {BEDFORM:{}}
                tic = time.time()
                assert os.path.exists(data_origin) == True
                data = dir_archive(data_origin, {}, serialized=True, cached=True)
                data.load("all_sx_maps", "all_sy_maps")

                if which_quantity == 0:
                    # derive the quantity you want to save -- for preventing OOM error,
                    # do the cv2 processing in memory for half the frames, clear memory and repeat once
                    NUM_FRAMES = data['all_sy_maps'].shape[-1]

                    if which_quantity == 0:
                        _, bed_data[BEDFORM]['all_slopeAzi_maps']\
                                    = cv2.cartToPolar(data['all_sy_maps'][:,:,:NUM_FRAMES//2],\
                                                      data['all_sx_maps'][:,:,:NUM_FRAMES//2], \
                                                      angleInDegrees=True)    
                        # Accumulate the data-
                        accumulated_data.append(bed_data[BEDFORM]['all_slopeAzi_maps'])
                        # clear space
                        bed_data[BEDFORM]['all_slopeAzi_maps']  = None

                        # repeat but notice this time the indices have changed
                        _, bed_data[BEDFORM]['all_slopeAzi_maps']\
                                    = cv2.cartToPolar(data['all_sy_maps'][:,:,NUM_FRAMES//2:],\
                                                      data['all_sx_maps'][:,:,NUM_FRAMES//2:], \
                                                      angleInDegrees=True)  
                        # Accumulate the data-
                        accumulated_data.append(bed_data[BEDFORM]['all_slopeAzi_maps'])
                        # clear space
                        bed_data[BEDFORM]['all_slopeAzi_maps']  = None
                elif which_quantity == 1:
                    accumulated_data.append(data['all_sx_maps'])
                elif which_quantity == 2:
                    accumulated_data.append(data['all_sy_maps'])

                toc = time.time()
                print("time taken to process 1 subfolder of data  = ", toc-tic)

                del bed_data, data
                gc.collect()

            # concatenate all the data so far
            data = np.concatenate(accumulated_data, axis =2)
            accumulated_data = None

            # save the accumulated data
            dic = {QUANTITY:data,\
                   "PORTION":PORTION,\
                    'BEDFORM':BEDFORM, 'FLOW_SPEED':FLOW_SPEED,'SUBMERGENCE':SUBMERGENCE}
#             dic = {QUANTITY:data,\
#                    "PORTION":PORTION}
            
            # if filenot found error, that could mean the filename is too long!
            filename = '{}_QUANT{}_PORTION{}_{}FLOW_{}H_test{}.mat'.format\
                            (BEDFORM, QUANTITY, PORTION, FLOW_SPEED, SUBMERGENCE, TEST)
#             filename = '{}_QUANT{}_DIST{}cm_PORTION{}.mat'.format(BEDFORM, QUANTITY, DIST, PORTION)
#             filename = 'waterjet_powerHighCapped1_Dist0cm_QUANTITY_{}_PORTION{}.mat'.format(QUANTITY, PORTION)
            save_dest = os.path.join(save_location, filename)
            savemat(save_dest,dic)
            del dic, data
            gc.collect()
#     except:
#         print("got exception for PORTION {}".format(PORTION))
#         continue

---------------------*****corals_fastFlow_ShallowH****----------------------
--------------QUANTITY0: slope_Azi_angle---------
#####PORTION: 1#####
waterjet_powerHighCapped1_Dist0cmBoilHeadtoROITop_15fps_25Pump_24cmH_industrialHose_subFolder1
save folder location created.
time taken to process 1 subfolder of data  =  55.841572999954224
#####PORTION: 2#####
waterjet_powerHighCapped1_Dist0cmBoilHeadtoROITop_15fps_25Pump_24cmH_industrialHose_subFolder2
time taken to process 1 subfolder of data  =  63.37475943565369
#####PORTION: 3#####
waterjet_powerHighCapped1_Dist0cmBoilHeadtoROITop_15fps_25Pump_24cmH_industrialHose_subFolder3
time taken to process 1 subfolder of data  =  58.17923355102539
#####PORTION: 4#####
waterjet_powerHighCapped1_Dist0cmBoilHeadtoROITop_15fps_25Pump_24cmH_industrialHose_subFolder4
time taken to process 1 subfolder of data  =  62.0894935131073
--------------QUANTITY1: slope_x---------
#####PORTION: 1#####
waterjet_powerHighCapped1_Dist0cmBoilHeadtoROITop_15fps_25Pu